In [53]:
# @title Magyar Irodalomtörténet Bibliográfiája rekord ellenőrző {display-mode: "form"}

# @title Földrajzi nevek keresése {display-mode: "form"}

import requests

# Define the Wikidata API endpoint
api_endpoint = "https://itidata.abtk.hu/w/api.php"

# Define the Wikidata Query Service endpoint
wdqs_endpoint = "https://query.itidata.abtk.hu/proxy/wdqs/bigdata/namespace/wdq/sparql"

# Set the request headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
    "Accept": "application/json"
}

def get_itidata_item_id(label):
    # Set the request parameters
    params = {
        "action": "wbsearchentities",
        "format": "json",
        "language": "hu",
        "search": label,
        "type": "item",
    }

    try:
        # Send the HTTP request to the itidata API
        response = requests.get(api_endpoint, params=params)
        data = response.json()

        # Check if any entities were found
        if data.get("search"):
            # Get a list of itidata item IDs from the search results
            itidata_item_ids = [result["id"] for result in data["search"]]

            return itidata_item_ids

    except Exception as e:
        print(f"Error: {e}")

    # Return an empty string if an error occurred or no match found
    return ""

def get_label_json_property_itidata(item_id, what_do_yo_need):

    # Set the request parameters
    params = {
        "action": "wbgetentities",
        "ids": item_id,
        "languages": "hu|en",  # Specify languages for labels (Hungarian and English)
        "format": "json"
    }

    try:
        # Send the HTTP request to the Wikidata API
        response = requests.get(api_endpoint, params=params)
        data = response.json()

        # Extract relevant information from the response
        entity = data.get("entities", {}).get(item_id, {})
        labels = entity.get("labels", {})
        label_hu = labels.get("hu", {}).get("value", "")
        label_en = labels.get("en", {}).get("value", "")

        if what_do_yo_need == "json":
            return data
        elif what_do_yo_need.lstrip("P").isdigit():
            property_value = data["entities"][item_id]["claims"][what_do_yo_need][0]["mainsnak"]["datavalue"]["value"]["id"]
            return property_value
        else:
            return (label_hu, label_en)

    except Exception as e:
        print(f"Error: {e}")

    # Return empty strings if an error occurred
    return "Unknown"

def get_item_labels_from_itidata(item_id):

    # Set the request parameters
    params = {
        "action": "wbgetentities",
        "ids": item_id,
        "languages": "hu|en",  # Specify languages for labels (Hungarian and English)
        "format": "json"
    }

    try:
        # Send the HTTP request to the Wikidata API
        response = requests.get(api_endpoint, params=params)
        data = response.json()

        # Extract relevant information from the response
        entity = data.get("entities", {}).get(item_id, {})
        labels = entity.get("labels", {})
        label_hu = labels.get("hu", {}).get("value", "")
        label_en = labels.get("en", {}).get("value", "")

        if label_hu is None or label_hu == "":
            return label_en + f"({item_id})"
        else:
            return label_hu + f"({item_id})"

    except Exception as e:
        print(f"Error: {e}")

    # Return empty strings if an error occurred
    return "Unknown"


# print(get_itidata_subclasses_of_human_settlement())
# print(get_item_labels_from_itidata("Q2727"))
# bud_ids = get_itidata_item_id("Budapest")
# print(bud_ids)
# bud_dict = get_itidata_item_info(bud_ids)
# for key, value in bud_dict.items():
#     print(f"{key}: {value}")

mib_instance_of_dict = {"Q24": "szakcikk", "Q23": "cikk", "Q24104": "szakkönyv", "Q17": "könyv"}
periodicals_dict = {"Q18": "folyóirat", "Q19": "irodalmi lap", "Q308137": "hírlap", "Q20": "tudományos folyóirat", "Q22": "újság"}
itidata_input_id = ""

# Input itidata id 
while itidata_input_id.lstrip("Q").isdigit() is False:
    itidata_input_id = input("ITIdata entitás id [Q12345]: ")
itidata_json = get_label_json_property_itidata(itidata_input_id, "json")

quality_check = True

# Check Englis and Hungarian lables
try:
    if itidata_json["entities"][itidata_input_id]["labels"]["hu"]["value"] == "":
        print("Hiányzik a magyar nyelvű címke.")
        quality_check = False
    else:
        print("Magyar címke: ", itidata_json["entities"][itidata_input_id]["labels"]["hu"]["value"])
    if itidata_json["entities"][itidata_input_id]["labels"]["en"]["value"] == "":
        print("Hiányzik a magyar nyelvű címke.")
        quality_check = False
    else:
        print("Angol címke: ", itidata_json["entities"][itidata_input_id]["labels"]["en"]["value"])
except KeyError:
    print("Hiányzó címke.")
    quality_check = False

# Chech P1
try:
    instance_of = itidata_json["entities"][itidata_input_id]["claims"]["P1"][0]["mainsnak"]["datavalue"]["value"]["id"]
    if instance_of not in mib_instance_of_dict.keys():
        print(f"A P1 értéke ({get_item_labels_from_itidata(instance_of)}) nem megengedett! Lehetséges értékek: \n"
          f"{[value[key] for key, value in mib_instance_of_dict.items()]}")
        quality_check = False
    else:
        print(f"Osztály, amelynek példánya: {mib_instance_of_dict[instance_of]} ({instance_of})")

        # Periodicals
        if instance_of == "Q23" or instance_of == "Q24":
           
            # Published in
            try:
                published_in = itidata_json["entities"][itidata_input_id]["claims"]["P44"][0]["mainsnak"]["datavalue"]["value"]["id"]
                print("Mű, amelyben megjelent (P44): ", get_item_labels_from_itidata(published_in))
                if get_label_json_property_itidata(published_in, "P1") not in periodicals_dict.keys():
                    print(f"A mű, amnelyben megjelent (P44) nem periodika. \n"
                          f"Lehetséges értékek: {[(key, value) for key, value in periodicals_dict.items()]}")
                    quality_check = False
            except KeyError:
                print("Hiányzik a mű, amelyben megjelent (P44) állítás.")
                quality_check = False
           
           # Volume
            try:
                volume = itidata_json["entities"][itidata_input_id]["claims"]["P45"][0]["mainsnak"]["datavalue"]["value"]
                if volume.strip() == "":
                    print("Az évfolyam, kötet (P45) értéke üres.")
                    quality_check = False
                else:
                    print("Évfolyam, kötet (P45)", volume)
            except KeyError:
                print("Hiányzik az évfolyam, kötet (P45)")
                quality_check = False
            
except KeyError:
    print("Hiányzik az Osztály amelynek példánya (P1) értéke. Lehetséges értékek: \n"
          "szakcikk (Q24), cikk (Q23), szakkönyv (Q24104), könyv (Q17)")
    quality_check = False

# Check MIB as collection
# TODO: What if the record is part of multiple collections?
try:
    collection = itidata_json["entities"][itidata_input_id]["claims"]["P17"][0]["mainsnak"]["datavalue"]["value"]["id"]
    if collection != "Q2138":
        print(f"A Gyűjtemény (P17) értéke ({collection}) hibás. Helyes érték: A magyar irodalomtörténet bibliográfiája (Q2138)")
        quality_check = False
except KeyError:
    print("Hiányzik: Gyűjtemény (P17): A magyar irodalomtörténet bibliográfiája (Q2138).")
    quality_check = False

# Check genre
try: 
    genre = itidata_json["entities"][itidata_input_id]["claims"]["P41"][0]["mainsnak"]["datavalue"]["value"]["id"]
    print("Műfaj: ", get_item_labels_from_itidata(genre))
except KeyError:
    print("Hiányzik a műfaj (P41).")
    quality_check = False

# Check author entity    
try:
    author_id = itidata_json["entities"][itidata_input_id]["claims"]["P7"][0]["mainsnak"]["datavalue"]["value"]["id"]
    print("Szerző entitás: ", get_item_labels_from_itidata(author_id))
    if get_label_json_property_itidata(author_id, "P1") != "Q2":
        print("A szerző entitás osztálya (P1) nem ember (Q2).")
        quality_check = False
except KeyError:
    print("Hiányzó szerző entitás (P7)")
    quality_check = False
    


if quality_check:
    print("Ellenőrző kód: ", hash(str(itidata_json)))

Magyar címke:  Kazinczy, a dokumentátor
Angol címke:  Kazinczy, a dokumentátor
Osztály, amelynek példánya: szakcikk (Q24)
Mű, amelyben megjelent (P44):  Széphalom(Q23946)
Évfolyam, kötet (P45) 22.
Hiányzik a műfaj (P41).
Hiányzó szerző entitás (P7)
